In [1]:
from fastai.vision import *
import time 

def read_file():
    path = Path('../../')
    csv = path/"label_list.csv"
    art = pd.read_csv(csv, sep="\t")
    
    return art, path

art, path = read_file()

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3242: DtypeWarning: Columns (3,5,6,7,10,12,13,15,17,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [2]:
def filter_art():
    filtered = art.groupby("Style").filter(lambda x: x['Style'].count() > 500)
    filtered = filtered.sample(frac=1, random_state=42).groupby("Style").head(1000)
    filtered = filtered[filtered["Style"] != "Arita ware"]
    filtered = filtered[filtered["Style"] != "Sawankhalok ware"]
    return filtered

filtered = filter_art()
print(filtered.groupby("Style")['Style'].count())
filtered["image_url"]

Style
Abstract Art                    1000
Abstract Expressionism          1000
Academicism                     1000
Art Deco                         880
Art Informel                    1000
Art Nouveau (Modern)            1000
Baroque                         1000
Color Field Painting            1000
Conceptual Art                   984
Concretism                       733
Contemporary Realism             862
Cubism                          1000
Early Renaissance               1000
Expressionism                   1000
Fauvism                          667
Hard Edge Painting               534
High Renaissance                1000
Impressionism                   1000
Ink and wash painting            514
Lyrical Abstraction              807
Magic Realism                   1000
Mannerism (Late Renaissance)    1000
Minimalism                      1000
Naïve Art (Primitivism)         1000
Neo-Expressionism                753
Neo-Romanticism                  584
Neoclassicism                   

703568    https://uploads6.wikiart.org/images/gianni-pia...
686790    https://uploads7.wikiart.org/images/diego-vela...
651365    https://uploads8.wikiart.org/images/claude-mon...
708578    https://uploads4.wikiart.org/images/claude-via...
729602    https://uploads3.wikiart.org/images/balthus/la...
                                ...                        
703516    https://uploads5.wikiart.org/images/antonio-pa...
726305    https://uploads6.wikiart.org/images/emil-schum...
716461    https://uploads8.wikiart.org/images/ludwig-wil...
741735    https://uploads2.wikiart.org/images/erte/lucre...
739517    https://uploads0.wikiart.org/images/marcelle-c...
Name: image_url, Length: 35354, dtype: object

In [11]:
def to_basename(x):
    x.Path = os.path.basename(x.Path)
    
    return x

def download_images():
    dls = filtered[["image_url", "Path", "Style"]]

    dls = filtered.apply(to_basename, axis=1)
    
    i = 0
    handle = display(i, display_id=True)
    print('of')
    display(dls.shape[0])

    for item in dls.itertuples():
        i = i + 1
        handle.update(i)
        download_url(item.image_url, path/"images"/item.Path, overwrite=False, show_progress=False)
    
    return dls
        
dls = download_images()

35354

of


35354

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
def create_and_save_label_list():
    label_list = dls[["Path", "Style"]].rename(columns={"Path": "name", "Style": "label"})
    label_list.to_csv(path/"images"/'labels.csv', index=False)
    return label_list

label_list = create_and_save_label_list()
label_list

,name,label
703568,Silver Wing With Crown 1972.jpg,Minimalism
686790,The Virgin Appearing To St Ildephonsus And Giv...,Baroque
651365,The Japanese Bridge 1924.jpg,Impressionism
708578,Untitled No 59 2002.jpg,Color Field Painting
729602,Landscape With Oxen 1942.jpg,Expressionism
...,...,...
703516,Untitled 1.jpg,Neo-Expressionism
726305,Gg 10 1991.jpg,Tachisme
716461,Interferenz Relief 1975.jpg,Op Art
741735,Lucrezia Bori Pelleas And Melisande.jpg,Art Deco
